# 1. File Loading

In [1]:
import torch
import pandas as pd
import numpy as np
import os

# load data
from torch.utils.data import DataLoader
import multiprocessing as mp
import time

# train
from torch import nn
from torch.nn import functional as F
import torch.nn.init

# visualization
import matplotlib.pyplot as plt
from tensorboardX import SummaryWriter

In [2]:
batch_size = 64
num_workers = 40
writer = SummaryWriter('runs/MFCC')
train_set = 'slim_train_set3.csv'
test_set = 'slim_test_set3.csv'

In [3]:
def read_csv(file_name):
    df = pd.read_csv(file_name, header=None)
    return df

file_list = []
for i in range(26):
    file_list.append('./separated data1/'+str(i+1)+'.csv')

In [4]:
class MFCCDataset(torch.utils.data.Dataset):
    def __init__(self, file_list):
        pool = mp.Pool(processes = 100)
        start = time.time()
        self.raw = pd.concat(pool.map(read_csv, file_list))
        print("time :", time.time() - start)
        pool.close()
        pool.join()
        
        self.label = torch.IntTensor(np.array(self.raw[0].values).reshape(len(self.raw[0].values), 1))
        self.len = len(self.label)
        self.data = torch.Tensor(np.array(self.raw.loc[:,1:]).reshape(len(self.label),100,500))

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        return self.data[idx], self.label[idx]

print("Loading data...")

dataset = MFCCDataset(file_list)

dataloader = torch.utils.data.DataLoader(
   dataset, batch_size = batch_size, num_workers = num_workers, drop_last=True
)
print("data is ready!")

Loading data...
time : 12.293453931808472
data is ready!


# 2. Neural Network

In [5]:
import random
USE_CUDA = torch.cuda.is_available() # GPU를 사용가능하면 True, 아니라면 False를 리턴
device = torch.device("cuda" if USE_CUDA else "cpu") # GPU 사용 가능하면 사용하고 아니면 CPU 사용\
print("다음 기기로 학습합니다:", device)
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

다음 기기로 학습합니다: cuda


In [6]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIn shape=(?, 28, 28, 1) -> 1, 100, 1000, 1
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=13, stride=1, padding=6),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=10, stride=10))

        # 두번째층
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=7, stride=1, padding=3),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=5, stride=5))

        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(1 * 5 * 128, 3, bias=True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

        dropout = torch.nn.Dropout(p=0.2)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out

In [7]:
# CNN 모델 정의
learning_rate = 0.001
training_epochs = 50
model = CNN()   #.to(device)
model = torch.nn.DataParallel(model)
model.cuda()
criterion = torch.nn.CrossEntropyLoss() #.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
total_batch = len(dataloader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 40


In [8]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in dataloader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.reshape(64,1,100,500).to(device)
        Y = Y.reshape(64,1)[:,0].to(device, dtype=torch.int64)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

        writer.add_scalar('training loss', avg_cost, epoch)
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 1.12650645
[Epoch:    2] cost = 1.05538595
[Epoch:    3] cost = 0.944598675
[Epoch:    4] cost = 0.752222538
[Epoch:    5] cost = 0.656351984
[Epoch:    6] cost = 0.591132045
[Epoch:    7] cost = 0.534315407
[Epoch:    8] cost = 0.502093852
[Epoch:    9] cost = 0.470042914
[Epoch:   10] cost = 0.440261364
[Epoch:   11] cost = 0.417840987
[Epoch:   12] cost = 0.401701421
[Epoch:   13] cost = 0.388154447
[Epoch:   14] cost = 0.387734532
[Epoch:   15] cost = 0.365888417
[Epoch:   16] cost = 0.369008601
[Epoch:   17] cost = 0.348668098
[Epoch:   18] cost = 0.324021786
[Epoch:   19] cost = 0.337027013
[Epoch:   20] cost = 0.382010281
[Epoch:   21] cost = 0.381545156
[Epoch:   22] cost = 0.325781822
[Epoch:   23] cost = 0.291062802
[Epoch:   24] cost = 0.283700019
[Epoch:   25] cost = 0.295023412
[Epoch:   26] cost = 0.276942998
[Epoch:   27] cost = 0.283612102
[Epoch:   28] cost = 0.27430293
[Epoch:   29] cost = 0.270522803
[Epoch:   30] cost = 0.255673021
[Epoch:   31]

In [9]:
print("Loading testset...")

file_list = []
for i in range(7):
    file_list.append('./separated data1/test'+str(i+1)+'.csv')

testset = MFCCDataset(file_list)

dataloader = torch.utils.data.DataLoader(
   testset, batch_size = batch_size, num_workers = num_workers, drop_last=True
)
print("testset is ready!")

Loading testset...
time : 7.3628692626953125
testset is ready!


In [10]:
for epoch in range(50,training_epochs+50):

    for X, Y in dataloader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.reshape(64,1,100,500).to(device)
        Y = Y.reshape(64,1)[:,0].to(device, dtype=torch.int64)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

        writer.add_scalar('training loss', avg_cost, epoch)
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:   51] cost = 0.271571994
[Epoch:   52] cost = 0.391798377
[Epoch:   53] cost = 0.483482569
[Epoch:   54] cost = 0.555835247
[Epoch:   55] cost = 0.615623593
[Epoch:   56] cost = 0.666564643
[Epoch:   57] cost = 0.710640013
[Epoch:   58] cost = 0.754226148
[Epoch:   59] cost = 0.800983548
[Epoch:   60] cost = 0.850287199
[Epoch:   61] cost = 0.899338186
[Epoch:   62] cost = 0.948819816
[Epoch:   63] cost = 1.02192807
[Epoch:   64] cost = 1.11935127
[Epoch:   65] cost = 1.18605697
[Epoch:   66] cost = 1.23221588
[Epoch:   67] cost = 1.26227736
[Epoch:   68] cost = 1.28833115
[Epoch:   69] cost = 1.30190921
[Epoch:   70] cost = 1.30999029
[Epoch:   71] cost = 1.31609857
[Epoch:   72] cost = 1.32089901
[Epoch:   73] cost = 1.32476759
[Epoch:   74] cost = 1.32816887
[Epoch:   75] cost = 1.33116043
[Epoch:   76] cost = 1.33384609
[Epoch:   77] cost = 1.33629143
[Epoch:   78] cost = 1.33854115
[Epoch:   79] cost = 1.34062386
[Epoch:   80] cost = 1.34255981
[Epoch:   81] cost = 1.34436

In [12]:
with torch.no_grad():
    X_test = testset.data.view(len(testset), 1, 100, 500).float().to(device)
    Y_test = testset.label.to(device, dtype=torch.int64)[:,0]

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9999999403953552
